<h1 style="text-align:center;">Change Boot Order</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contains a example code to demonstrate how to change boot order for HPE iLO systems. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
        <li>getpass - to take password as inputs securely</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our change boot order function overview, in the end this will change the boot order by one position.</h3><br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings.<br>

1. This function initializes bios_boot_uri and bios_boot_response variables to None.
<br>
    
2. Call get_resource_directory() function by passing the "_redfishobj" as parameter to retrieve the resource directory of the server. If resource directory is not available or it is disabled, find the relevant URI by navigating through the server's API. If resource directory is available, find the relevant URI from the resource directory.
    <br>

3. Then we navigate through the hirerachy to find <i>bios_boot_uri</i>. If bios_boot_uri is not None, retrieve the BIOS boot settings using get() method. Then Retrieve the URI for BIOS boot settings from the response.

    <br>

4. If bios_password is provided, update the BIOS password with it.
    <br>

5. Print the current boot order. Rotate the first boot device to the end of the boot order. Update the boot order using the patch() method with the new boot order.
    <br>

6. If the HTTP response status code is not 200 or 201, print an error message. If the HTTP response status code is 400, print the iLO extended info error message. If the HTTP response status code is 200 or 201, print a success message and the updated BIOS boot settings.
    
    <br>

7. If bios_boot_uri is None, print an error message.
    

 </div>

In [ ]:
def change_boot_order(_redfishobj, bios_password):

    bios_boot_uri = None
    bios_boot_response = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        systems_uri = _redfishobj.root.obj['Systems']['@odata.id']
        systems_response = _redfishobj.get(systems_uri)
        systems_members_uri = next(iter(systems_response.obj['Members']))['@odata.id']
        systems_members_response = _redfishobj.get(systems_members_uri)
        bios_uri = systems_members_response.obj['Bios']['@odata.id']
        bios_response = _redfishobj.get(bios_uri)
        bios_boot_uri = bios_response.obj.Oem.Hpe.Links.Boot['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeServerBootSettings.' in instance['@odata.type']:
                bios_boot_uri = instance['@odata.id']
                break

    if bios_boot_uri:
        bios_boot_response = _redfishobj.get(bios_boot_uri)
        #Bios boot settings URI is needed
        bios_boot_settings_uri = bios_boot_response.obj['@Redfish.Settings']['SettingsObject']['@odata.id']
        #update BIOS password
        if bios_password:
            _redfishobj.bios_password = bios_password
        sys.stdout.write("Rotating the first boot device to the end of the boot order.\n")
        sys.stdout.write('Current Order:\n')
        boot_order = bios_boot_response.obj['DefaultBootOrder']
        for indx, boot_device in enumerate(boot_order):
            sys.stdout.write('Pos ' + str(indx) + ' : ' + boot_device + '\n')
        device = boot_order.pop(0)
        sys.stdout.write("Rotating device: \'%s\' to the end of the boot order.\n" % device)
        boot_order.append(device)

        body = {'DefaultBootOrder': boot_order}
        resp = _redfishobj.patch(bios_boot_settings_uri, body)

        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success! Your system may need to be restarted.\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
    else:
        sys.stderr.write("Unable to find Boot Order URI.\n")

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the boot order.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>change_boot_order</b> function with our redfish object and BIOS Password as parameters<br>
    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    BIOS_PASSWORD = None
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    change_boot_order(REDFISHOBJ, BIOS_PASSWORD)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help please check the follwoing links for reference:
    <br>
    1. Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">Python ilorest library</a><br>
    2. HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">ilorestful api explorer</a><br>
    3. HPE iLO Redfish Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/">HPE iLO Redfish API Documentation</a><br>
    4. REST API: <a href = "https://restfulapi.net/">Restful API</a><br>    
</div>